# Main Freeglisse Scrapping Code

## Setup and Configuration

In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

## Data collection process

### Function to get all URLs products

In [4]:
def get_all_product_urls(base_url):
    all_product_urls = []
    current_page = 1
    while True:
        # Build the current page URL by adding the pagination parameter
        url = f"{base_url}?page={current_page}"
        response = requests.get(url)
        # Break the loop if the page request was unsuccessful
        if response.status_code != 200:
            break 

        soup = BeautifulSoup(response.content, 'html.parser')
        products = soup.find_all('div', class_='product')

        # Find the URL for each product and add it to the list
        page_product_urls = [p.find('a')['href'] for p in products if p.find('a')]
        if not page_product_urls:
            break  # Stop if no product URLs are found on the page

        all_product_urls.extend(page_product_urls)

        # Check if there is a next page
        next_button = soup.find('a', rel='next')
        if not next_button or 'disabled' in next_button.get('class', []):
            break  # Stop if there is no 'next' button or if it's disabled
        
        current_page += 1  # Increment the page number
        
    return all_product_urls


# URL of the first page of products
base_url = 'https://freeglisse.com/fr/12-ski-occasion'
product_urls = get_all_product_urls(base_url)



In [5]:
def scrape_product_reviews(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    reviews = []
    reviews_container = soup.find_all('div', class_='netreviews_review_part')
    for review in reviews_container:
        comment = review.find('p', class_='netreviews_customer_review').text.strip()
        reviews.append(comment)
    return reviews

### Function to extract all datas we need of a single page

In [6]:
def scrape_product_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract canonical link
    canonical_link = soup.find('link', rel='canonical')['href']

    # Extract reference
    reference = soup.find('label', class_='label title-cate').find_next('span').text

    # Extract title
    title = soup.find('h1', class_='h1 product-detail-name').text
    
    # Extract the brand from the 'alt' attribute of the img
    brand_tag = soup.find('img', class_='img img-thumbnail manufacturer-logo')
    brand = brand_tag['alt'] if brand_tag else None

    # Extract the price excluding taxes
    pretax_price_meta = soup.find('meta', property='product:pretax_price:amount')
    pretax_price = pretax_price_meta['content'] if pretax_price_meta else None

    # Extract usual price and current price
    regular_price_tag = soup.find('span', class_='regular-price')
    current_price = soup.find('span', class_='current-price-value').text.strip().replace('\xa0', '')

    # Extract availability information
    available = soup.find('span', id='availability_message').text

    # Extract qualities
    qualities = [span.text for span in soup.find_all('span', class_='radio-label') 
                if not span.find_previous_sibling('input', type='radio').has_attr('disabled')]

    # Extract product weight
    product_weight_meta = soup.find('meta', property='product:weight:value')
    product_weight = product_weight_meta['content'] if product_weight_meta else None

    # Extract product reviews
    product_reviews = scrape_product_reviews(url)
    
    # Extract the number given by stars
    reviews_count_1_star = soup.find('span', class_='netreviews_rate_total1').text.strip() if soup.find('span', class_='netreviews_rate_total1') else '0'
    reviews_count_2_stars = soup.find('span', class_='netreviews_rate_total2').text.strip() if soup.find('span', class_='netreviews_rate_total2') else '0'
    reviews_count_3_stars = soup.find('span', class_='netreviews_rate_total3').text.strip() if soup.find('span', class_='netreviews_rate_total3') else '0'
    reviews_count_4_stars = soup.find('span', class_='netreviews_rate_total4').text.strip() if soup.find('span', class_='netreviews_rate_total4') else '0'
    reviews_count_5_stars = soup.find('span', class_='netreviews_rate_total5').text.strip() if soup.find('span', class_='netreviews_rate_total5') else '0'

    # Extract the general note
    note_generale = None
    note_generale_tag = soup.find('p', class_='netreviews_note_generale')
    if note_generale_tag:
        note_generale_text = note_generale_tag.text.strip()
        if note_generale_text:
            note_generale = note_generale_text.split()[0] + "/5"

    # Extract the number of reviews
    nombre_avis = None
    review_count_tag = soup.find('span', id='reviewCount')
    if review_count_tag:
        review_count_text = review_count_tag.text.strip()
        if re.match(r'^\d+$', review_count_text):
            nombre_avis = int(review_count_text)

    # Extract product details
    product_details = {}
    details = soup.find_all(['dt', 'dd'])
    for i in range(0, len(details), 2):
        if details[i].get('class', [''])[0] == 'name' and details[i + 1].get('class', [''])[0] == 'value':
            product_details[details[i].text.strip()] = details[i + 1].text.strip()

    # Extract URL from first product image
    image_urls_tag = soup.find('img', class_='img img-thumb')
    image_urls = image_urls_tag['src'] if image_urls_tag else None


    # Build and return a dictionary of product information
    return {
        'title': title,
        'brand': brand,
        'regular_price': regular_price_tag,
        'current_price': current_price,
        'details': product_details,
        'available': available,
        'reference': reference,
        'qualities': qualities,
        'link': canonical_link,
        'weight': product_weight,
        'general_rate': note_generale,
        'nb_ratings': nombre_avis,
        'reviews': product_reviews,
        'image_urls': image_urls,
        'pretax_price': pretax_price,
        'reviews_count_1_star': reviews_count_1_star,
        'reviews_count_2_stars': reviews_count_2_stars,
        'reviews_count_3_stars': reviews_count_3_stars,
        'reviews_count_4_stars': reviews_count_4_stars,
        'reviews_count_5_stars': reviews_count_5_stars,
    }


### Extract datas of all pages

In [7]:
# URL of the first page of the products
base_url = 'https://freeglisse.com/fr/12-ski-occasion'
product_urls = get_all_product_urls(base_url)

# List to store details of all products
all_product_details = []

# Browse each product URL and retrieve details
for url in product_urls:
    details = scrape_product_page(url)
    all_product_details.append(details)

all_product_details

[{'title': 'Ski occasion Rossignol Sender 104 Ti 2023  + Fixations',
  'brand': 'Rossignol',
  'regular_price': <span class="regular-price">419,00 €</span>,
  'current_price': '335,20€',
  'details': {'Type': 'Freeride',
   'Utilisateur': 'Mixte',
   'Niveau': 'Performant',
   'Coloris': 'Gris',
   'Utilisateur - Configurateur': 'Freerideur adulte',
   'Economie de CO2 pour la planète (en kg)': '3.6',
   'Type de produit': 'Ski occasion freeride'},
  'available': 'Disponible',
  'reference': '18916_m27',
  'qualities': ['Qualité A',
   '164 cm',
   '172 cm',
   '178 cm',
   '186 cm',
   'Qualité A',
   '164 cm',
   '172 cm',
   '178 cm',
   '186 cm'],
  'link': 'https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html',
  'weight': '6.000000',
  'general_rate': '5/5',
  'nb_ratings': 1,
  'reviews': ["Hormis le problème de fixations après avoir pris le soins de les lustré préparé les cares et fartage la notation 

### Convert result into a Dataframe

In [8]:
df = pd.DataFrame(all_product_details)
df.head()

,title,brand,regular_price,current_price,details,available,reference,qualities,link,weight,general_rate,nb_ratings,reviews,image_urls,pretax_price,reviews_count_1_star,reviews_count_2_stars,reviews_count_3_stars,reviews_count_4_stars,reviews_count_5_stars
0,Ski occasion Rossignol Sender 104 Ti 2023 + F...,Rossignol,"[419,00 €]","335,20€","{'Type': 'Freeride', 'Utilisateur': 'Mixte', '...",Disponible,18916_m27,"[Qualité A, 164 cm, 172 cm, 178 cm, 186 cm, Qu...",https://freeglisse.com/fr/ski-occasion-adulte-...,6.000000,5/5,1.0,[Hormis le problème de fixations après avoir p...,https://freeglisse.com/70892-small_default/ski...,279.333334,0,0,0,0,1
1,Ski occasion Rossignol Sender 94 Ti 2023 + Fi...,Rossignol,"[359,00 €]","287,20€","{'Type': 'Freeride', 'Utilisateur': 'Mixte', '...",Disponible,18915_i26,"[Qualité A, 156 cm, 164 cm, 172 cm, Qualité A,...",https://freeglisse.com/fr/ski-occasion-adulte-...,6.000000,4/5,1.0,[good cuality],https://freeglisse.com/70897-small_default/ski...,239.333334,0,0,0,1,0
2,Ski de fond occasion Rossignol LTS Junior + fi...,Rossignol,None,"19,00€","{'Type': 'Alternatif', 'Utilisateur': 'Junior'...",Disponible,18974_mz_l18_violet,"[Qualité C, 150 cm, 160 cm, 170 cm, Qualité C,...",https://freeglisse.com/fr/ski-de-fond-occasion...,4.000000,None,NaN,[],None,15.833333,0,0,0,0,0
3,Ski de fond occasion Toutes marques + fixation...,Toutes marques,None,"15,00€","{'Type': 'Alternatif', 'Utilisateur': 'Mixte',...",Disponible,15468_mz_l20,"[Qualité C, 140 cm, 150 cm, 160 cm, 170 cm, Qu...",https://freeglisse.com/fr/ski-de-fond-occasion...,4.000000,4.8/5,10.0,[Je n'ai pas encore pu les tester car pas de c...,https://freeglisse.com/43792-small_default/ski...,12.5,0,0,0,2,8
4,Ski occasion Rossignol Nova 6 + fixations,Rossignol,None,"169,00€","{'Type': 'Piste', 'Utilisateur': 'Femme', 'Niv...",Disponible,18890_l36,"[Qualité A, Qualité B, Qualité C, 149 cm, Qual...",https://freeglisse.com/fr/ski-occasion-femme-l...,6.000000,3.2/5,5.0,"[Etat du produit conforme à mes attentes!, Gew...",https://freeglisse.com/70606-small_default/ski...,140.833333,2,0,0,1,2


## Data cleaning and preparation

In [ ]:
# Normalize JSON data into a flat table
details = pd.json_normalize(df['details'])

In [10]:
df = df[['link', 'reference', 'brand', 'title', 'regular_price','current_price', 'available', 'qualities', 'weight', 'general_rate', 'nb_ratings', 'reviews', 'image_urls', 'pretax_price', 'reviews_count_1_star', 'reviews_count_2_stars', 'reviews_count_3_stars', 'reviews_count_4_stars', 'reviews_count_5_stars']]

In [11]:
# Concat details to main dataframe
df = pd.concat([df, details], axis=1)

In [12]:
df = df.rename(columns={'current_price': 'price'})

In [13]:
def extract_qualities_and_sizes(qualities_list):
    quality_pattern = re.compile(r'Qualité [A-Z]')
    size_pattern = re.compile(r'\d+ cm')

    available_qualities = [q for q in qualities_list if quality_pattern.match(q)]
    sizes = [s for s in qualities_list if size_pattern.match(s)]

    # Elimination of duplicates for qualities
    available_qualities = list(set(available_qualities))
    sizes = list(set(sizes))
    
    return available_qualities, sizes

# Applying the function to each row of the dataframe
df['available_qualities'], df['sizes'] = zip(*df['qualities'].apply(extract_qualities_and_sizes))

# Display for verification
df['available_qualities'].head(), df['sizes'].head()


(0                          [Qualité A]
 1                          [Qualité A]
 2                          [Qualité C]
 3                          [Qualité C]
 4    [Qualité C, Qualité B, Qualité A]
 Name: available_qualities, dtype: object,
 0    [164 cm, 172 cm, 186 cm, 178 cm]
 1            [164 cm, 172 cm, 156 cm]
 2            [160 cm, 170 cm, 150 cm]
 3    [160 cm, 140 cm, 170 cm, 150 cm]
 4                            [149 cm]
 Name: sizes, dtype: object)

In [14]:
df = df.drop(columns='qualities')
df.head()

,link,reference,brand,title,regular_price,price,available,weight,general_rate,nb_ratings,...,reviews_count_5_stars,Type,Utilisateur,Niveau,Coloris,Utilisateur - Configurateur,Economie de CO2 pour la planète (en kg),Type de produit,available_qualities,sizes
0,https://freeglisse.com/fr/ski-occasion-adulte-...,18916_m27,Rossignol,Ski occasion Rossignol Sender 104 Ti 2023 + F...,"[419,00 €]","335,20€",Disponible,6.000000,5/5,1.0,...,1,Freeride,Mixte,Performant,Gris,Freerideur adulte,3.6,Ski occasion freeride,[Qualité A],"[164 cm, 172 cm, 186 cm, 178 cm]"
1,https://freeglisse.com/fr/ski-occasion-adulte-...,18915_i26,Rossignol,Ski occasion Rossignol Sender 94 Ti 2023 + Fi...,"[359,00 €]","287,20€",Disponible,6.000000,4/5,1.0,...,0,Freeride,Mixte,Performant,Noir,Freerideur adulte,3.6,Ski occasion freeride,[Qualité A],"[164 cm, 172 cm, 156 cm]"
2,https://freeglisse.com/fr/ski-de-fond-occasion...,18974_mz_l18_violet,Rossignol,Ski de fond occasion Rossignol LTS Junior + fi...,None,"19,00€",Disponible,4.000000,None,NaN,...,0,Alternatif,Junior,Loisir,Violet,NaN,3.6,Ski de fond occasion alternatif norme SNS,[Qualité C],"[160 cm, 170 cm, 150 cm]"
3,https://freeglisse.com/fr/ski-de-fond-occasion...,15468_mz_l20,Toutes marques,Ski de fond occasion Toutes marques + fixation...,None,"15,00€",Disponible,4.000000,4.8/5,10.0,...,8,Alternatif,Mixte,Loisir,Blanc,NaN,3.6,Ski de fond occasion alternatif norme SNS,[Qualité C],"[160 cm, 140 cm, 170 cm, 150 cm]"
4,https://freeglisse.com/fr/ski-occasion-femme-l...,18890_l36,Rossignol,Ski occasion Rossignol Nova 6 + fixations,None,"169,00€",Disponible,6.000000,3.2/5,5.0,...,2,Piste,Femme,Loisir,Violet,une femme,3.6,Ski occasion femme loisir,"[Qualité C, Qualité B, Qualité A]",[149 cm]


In [15]:
df['price'] = df['price'].str.replace(',', '.').str.replace('€', '').astype(float)

In [16]:
df['regular_price'] = df['regular_price'].astype(str)

In [17]:
df['regular_price'] = (
    df['regular_price']
    .str.extract(r'(\d+[\.,]?\d*)')[0]  # Extract numbers with a period or comma as decimal separator.
    .str.replace(',', '.')  # Replace the comma with a decimal point.
    .astype(float)  # Convert to float.
)


In [18]:
df['regular_price'] = df['regular_price'].replace(5, np.nan)

In [19]:
df['weight'] = pd.to_numeric(df['weight'], errors='coerce')
df['weight'] = df['weight'].round()

In [20]:
df = df.rename(columns={'Type': 'type', 
                        'Utilisateur': 'user', 
                        'Niveau': 'level', 
                        'Utilisateur - Configurateur': 'user_config', 
                        'Economie de CO2 pour la planète (en kg)': 'eco_co2',
                        'Type de produit': 'product_type',
                        'available_qualities': 'qualities',
                        'Coloris': 'color'
                        })

In [21]:
df['reference'] = df['reference'].str.slice(0, 5)

In [22]:
df.columns

Index(['link', 'reference', 'brand', 'title', 'regular_price', 'price',
       'available', 'weight', 'general_rate', 'nb_ratings', 'reviews',
       'image_urls', 'pretax_price', 'reviews_count_1_star',
       'reviews_count_2_stars', 'reviews_count_3_stars',
       'reviews_count_4_stars', 'reviews_count_5_stars', 'type', 'user',
       'level', 'color', 'user_config', 'eco_co2', 'product_type', 'qualities',
       'sizes'],
      dtype='object')

In [23]:
columns_order = ['link', 'reference', 'brand', 'title', 'available', 'pretax_price', 'regular_price', 'price', 'type', 'user', 'level', 'color',
        'user_config', 'eco_co2', 'product_type', 'qualities', 'sizes', 'weight', 'general_rate', 'nb_ratings', 'reviews',
        'reviews_count_1_star', 'reviews_count_2_stars', 'reviews_count_3_stars', 'reviews_count_4_stars', 'reviews_count_5_stars', 'image_urls']
df = df[columns_order]

In [24]:
df['pretax_price'] = round(df['pretax_price'].astype(float), 2)

In [25]:
df['reviews_count_1_star'] = df['reviews_count_1_star'].astype(int)
df['reviews_count_2_stars'] = df['reviews_count_2_stars'].astype(int)
df['reviews_count_3_stars'] = df['reviews_count_3_stars'].astype(int)
df['reviews_count_4_stars'] = df['reviews_count_4_stars'].astype(int)
df['reviews_count_5_stars'] = df['reviews_count_5_stars'].astype(int)

### Creation of a new dataframe dedicated to ratings

In [26]:
columns_ratings = ['link', 'general_rate', 'nb_ratings', 'reviews_count_1_star', 'reviews_count_2_stars', 'reviews_count_3_stars',
                    'reviews_count_4_stars', 'reviews_count_5_stars', 'reviews']
df_ratings = df[columns_ratings]
df_ratings.head()

,link,general_rate,nb_ratings,reviews_count_1_star,reviews_count_2_stars,reviews_count_3_stars,reviews_count_4_stars,reviews_count_5_stars,reviews
0,https://freeglisse.com/fr/ski-occasion-adulte-...,5/5,1.0,0,0,0,0,1,[Hormis le problème de fixations après avoir p...
1,https://freeglisse.com/fr/ski-occasion-adulte-...,4/5,1.0,0,0,0,1,0,[good cuality]
2,https://freeglisse.com/fr/ski-de-fond-occasion...,None,NaN,0,0,0,0,0,[]
3,https://freeglisse.com/fr/ski-de-fond-occasion...,4.8/5,10.0,0,0,0,2,8,[Je n'ai pas encore pu les tester car pas de c...
4,https://freeglisse.com/fr/ski-occasion-femme-l...,3.2/5,5.0,2,0,0,1,2,"[Etat du produit conforme à mes attentes!, Gew..."


### Export ratings dataframe

In [27]:
df_ratings.to_csv('ratings.csv')

### Creation of a new dataframe dedicated to products details

In [28]:
columns_product = ['link', 'reference', 'brand', 'title', 'available', 'type', 'product_type', 'user', 
                    'level', 'user_config', 'color', 'sizes', 'weight',
                    'image_urls', 'price', 'regular_price', 'pretax_price', 'eco_co2']
df_product = df[columns_product]
df_product.head()

,link,reference,brand,title,available,type,product_type,user,level,user_config,color,sizes,weight,image_urls,price,regular_price,pretax_price,eco_co2
0,https://freeglisse.com/fr/ski-occasion-adulte-...,18916,Rossignol,Ski occasion Rossignol Sender 104 Ti 2023 + F...,Disponible,Freeride,Ski occasion freeride,Mixte,Performant,Freerideur adulte,Gris,"[164 cm, 172 cm, 186 cm, 178 cm]",6.0,https://freeglisse.com/70892-small_default/ski...,335.2,419.0,279.33,3.6
1,https://freeglisse.com/fr/ski-occasion-adulte-...,18915,Rossignol,Ski occasion Rossignol Sender 94 Ti 2023 + Fi...,Disponible,Freeride,Ski occasion freeride,Mixte,Performant,Freerideur adulte,Noir,"[164 cm, 172 cm, 156 cm]",6.0,https://freeglisse.com/70897-small_default/ski...,287.2,359.0,239.33,3.6
2,https://freeglisse.com/fr/ski-de-fond-occasion...,18974,Rossignol,Ski de fond occasion Rossignol LTS Junior + fi...,Disponible,Alternatif,Ski de fond occasion alternatif norme SNS,Junior,Loisir,NaN,Violet,"[160 cm, 170 cm, 150 cm]",4.0,None,19.0,NaN,15.83,3.6
3,https://freeglisse.com/fr/ski-de-fond-occasion...,15468,Toutes marques,Ski de fond occasion Toutes marques + fixation...,Disponible,Alternatif,Ski de fond occasion alternatif norme SNS,Mixte,Loisir,NaN,Blanc,"[160 cm, 140 cm, 170 cm, 150 cm]",4.0,https://freeglisse.com/43792-small_default/ski...,15.0,NaN,12.50,3.6
4,https://freeglisse.com/fr/ski-occasion-femme-l...,18890,Rossignol,Ski occasion Rossignol Nova 6 + fixations,Disponible,Piste,Ski occasion femme loisir,Femme,Loisir,une femme,Violet,[149 cm],6.0,https://freeglisse.com/70606-small_default/ski...,169.0,NaN,140.83,3.6


In [29]:
df.set_index('link')

,reference,brand,title,available,pretax_price,regular_price,price,type,user,level,...,weight,general_rate,nb_ratings,reviews,reviews_count_1_star,reviews_count_2_stars,reviews_count_3_stars,reviews_count_4_stars,reviews_count_5_stars,image_urls
link,,,,,,,,,,,,,,,,,,,,,
https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18916-ski-occasion-rossignol-sender-104-ti-2023-fixations.html,18916,Rossignol,Ski occasion Rossignol Sender 104 Ti 2023 + F...,Disponible,279.33,419.0,335.2,Freeride,Mixte,Performant,...,6.0,5/5,1.0,[Hormis le problème de fixations après avoir p...,0,0,0,0,1,https://freeglisse.com/70892-small_default/ski...
https://freeglisse.com/fr/ski-occasion-adulte-freeride-et-freestyle/18915-ski-occasion-rossignol-sender-94-ti-2023-fixations.html,18915,Rossignol,Ski occasion Rossignol Sender 94 Ti 2023 + Fi...,Disponible,239.33,359.0,287.2,Freeride,Mixte,Performant,...,6.0,4/5,1.0,[good cuality],0,0,0,1,0,https://freeglisse.com/70897-small_default/ski...
https://freeglisse.com/fr/ski-de-fond-occasion-alternatif-norme-sns/18974-ski-de-fond-occasion-rossignol-lts-junior-fixation-sns-profil.html,18974,Rossignol,Ski de fond occasion Rossignol LTS Junior + fi...,Disponible,15.83,NaN,19.0,Alternatif,Junior,Loisir,...,4.0,None,NaN,[],0,0,0,0,0,None
https://freeglisse.com/fr/ski-de-fond-occasion-alternatif-norme-sns/15468-ski-de-fond-occasion-toutes-marques-fixation-sns-profil.html,15468,Toutes marques,Ski de fond occasion Toutes marques + fixation...,Disponible,12.50,NaN,15.0,Alternatif,Mixte,Loisir,...,4.0,4.8/5,10.0,[Je n'ai pas encore pu les tester car pas de c...,0,0,0,2,8,https://freeglisse.com/43792-small_default/ski...
https://freeglisse.com/fr/ski-occasion-femme-loisir/18890-ski-occasion-rossignol-nova-6-fixations.html,18890,Rossignol,Ski occasion Rossignol Nova 6 + fixations,Disponible,140.83,NaN,169.0,Piste,Femme,Loisir,...,6.0,3.2/5,5.0,"[Etat du produit conforme à mes attentes!, Gew...",2,0,0,1,2,https://freeglisse.com/70606-small_default/ski...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://freeglisse.com/fr/ski-de-fond-occasion-alternatif-norme-sns/17488-ski-de-fond-occasion-junior-salomon-skin-race-fixation-sns-access.html,17488,Salomon,Ski de fond occasion junior Salomon Skin Race ...,Disponible,49.17,NaN,59.0,Alternatif,Junior,Loisir sport,...,4.0,None,NaN,[],0,0,0,0,0,None
https://freeglisse.com/fr/ski-occasion-adulte-all-mountain/16862-ski-occasion-lacroix-lx-gravity-fixations.html,16862,Lacroix,Ski occasion Lacroix LX Gravity + fixations,Rupture de stock,274.17,NaN,329.0,All mountain,Mixte,Performant,...,6.0,None,NaN,[],0,0,0,0,0,https://freeglisse.com/49174-small_default/ski...
https://freeglisse.com/fr/ski-occasion-adulte-performance/16493-ski-occasion-rossignol-react-8-fixations.html,16493,Rossignol,Ski occasion Rossignol React 8 + fixations,Rupture de stock,174.17,NaN,209.0,Piste,Mixte,Performant,...,6.0,4/5,1.0,[Correct],0,0,0,1,0,https://freeglisse.com/65942-small_default/ski...


### Export products dataframe

In [30]:
df.to_csv('products.csv')